# Qiskit Runtime

### Introduction

Qiskit Runtime은 작성된 양자 프로그램을 양자 시스템으로 보내고 계산이 수행되도록 하는 역할을 합니다. 이번 시간에는 IBM Quantum의 하드웨어와 거기에 엑세스 하는 **instance**에 접속하는 방법에 대해 알아보도록 하겠습니다. 계산을 수행하는데 걸릴 시간과 필요한 자원을 예측하고, session을 설정하고, 사용 시간을 예약하는 방법들도 자세히 살펴봅시다.

실행 단계의 흐름은 다음과 같습니다.
1. IBM Quantum Platform 또는 IBM Cloud 계정으로 원하는 instance에 접속합니다.
2. 사용할 backend를 선택합니다.
3. 양자 프로그램을 양자 시스템으로 전송합니다.
4. 실행 결과를 받고 확인합니다. 

# Run with primitives

### Introduction to primitives

컴퓨터 시스템은 여러 단계의 **추상화**(abstraction)가 쌓여 만들어집니다. 추상화는 우리가 해결해야 하는 문제와 직접적으로 관련된 부분에만 집중할 수 있도록 합니다. 하드웨어에 가까워질 수록 필요한 추상화의 단계는 낮아질 것이고 (예를 들면 전기 신호를 직접 조절하는 일), 반대로 더 복잡한 일을 수행하고 싶다면 더 높은 단계의 추상화가 필요할 것입니다 (예를 들어 프로그래밍 라이브러리를 활용해 덧셈을 하는 일).

이런 맥락에서, **primitive**란 가장 작은 프로세스 처리 단위이며, **주어진 추상화 단계**에 필요한 어떤 **유용한 것**을 만드는 가장 단순한 빌딩 블록입니다.

최근 양자컴퓨팅의 빠른 발전으로 높은 단계의 추상화에 대한 요구가 높아졌습니다. 우리가 더 큰 시스템과 더 복잡한 작업을 다루게 되면서, 각각의 큐비트의 신호를 다루는 것보다는 양자컴퓨터 전체를 어떤 작업을 수행하는 하나의 시스템으로 보는 것으로 관점이 옮겨지게 됩니다.

양자컴퓨터를 통해 수행하는 가장 일반적인 두 가지의 작업은 양자 상태를 샘플링하는 것과 기댓값을 계산하는 것입니다. 이 작업들은 두 가지의 Qiskit primitives: Sampler와 Estimator를 디자인하는 모티브가 되었습니다.

한 마디로, Qiskit primitives는 양자 프로그래밍을 현재의 고전 프로그래밍이 있는 곳으로 한 발짝 다가가게 하는 것이며, 하드웨어의 디테일 보다는 실제로 얻고 싶은 결과에 집중할 수 있도록 해줄 것입니다.

#### Implementation of Qiskit primitives

Qiskit primitives는 여러 서비스 제공자들이 각자의 Sampler와 Estimator를 구현할 수 있는 오픈소스 primitive 클래스로부터 정의됩니다. Qiskit을 이용한 구현 방법 중, 로컬 시뮬레이션을 primitive로 구현하는 가장 대표적인 예는 `qiskit.primitives` 모듈입니다. IBM의 Qiskit Runtime과 같은 서비스 제공자는 이러한 primitive를 구현함으로써 각자의 backend에 엑세스하는 적절한 방법을 제시할 수 있습니다.

#### Benefits of Qiskit primitives

Qiskit 유저들에게 primitive는 세세한 디테일에 신경 쓸 필요 없이 원하는 backend에서 작동하는 양자 코드를 쓸 수 있도록 해줍니다. 또 이러한 추상화를 통해 여러분은 여러분께 제공되는 하드웨어의 최고 성능에 더 쉽게 엑세스할 수도 있습니다. 예를 들어, Qiskit Runtime primitives를 통해 여러분은 `optimization_level`, `resilience_level`과 같은 옵션을 주는 것만으로 가장 최신의 error mitigation, error suppression 테크닉을 사용할 수 있습니다.

하드웨어 제공자들의 관점에서, primitive를 구현한다는 것은 여러분의 하드웨어 성능을 유저들에게 더 쉽게 제공할 수 있다는 것을 의미합니다. 여러분이 제공하는 하드웨어의 최고의 성능이 유저들에게 더 쉽게 다가갈 수 있는 것이죠.

#### Estimator

Estimator primitive는 양자 회로를 통해 만들어진 상태에서 특정 observable의 기댓값을 계산합니다. Estimator는 circuit과 observable의 쌍을 입력으로 받아서 (observable은 Pauli operator들의 가중합으로 표현됩니다), 각 circuit-observable 쌍에서 계산된 기댓값과 분산을 출력합니다. Estimator를 구현하는 방법에 따라 다양한 옵션을 지원하도록 만들 수도 있죠. Circuit은 변수를 포함할 수도 있으며, primitive의 입력을 통해 값을 설정할 수도 있습니다.

#### Sampler

Sampler primitive는 양자 회로를 실행하여 얻어지는 고전 레지스터의 출력값을 추출합니다. 이러한 이유로, Sampler의 입력값은 (변수를 포함할 수 있는) 양자 회로이며, 여기로부터 얻어지는 값의 준확률분포(quasi-probability distribution)을 출력합니다. 준확률분포는 확률분포와 유사하지만 error mitigation을 적용하는 과정에서 음수값이 나타날 수 있습니다.

#### How to use Qiskit primitives

`qiskit.primitives` 모듈은 primitive 스타일의 양자 프로그램 개발을 가능하게 하며, 특히 서로 다른 종류의 backend를 간단히 스위칭하여 사용할 수 있도록 디자인되었습니다. 이 모듈은 각 primitive 타입 당 세 가지의 클래스를 제공합니다:

1. `Sampler` and `Estimator`

이 클래스는 두 primitive의 대표적인 구현 방법이며, Qiskit의 built-in 시뮬레이터를 사용합니다. 이들은 Qiskit의 `quantum_info` 모듈을 사용하여 ideal한 statevector 시뮬레이션을 통해 결과를 얻습니다.

2. `BaseSampler` and `BaseEstimator`

이 클래스는 primitive를 구현하는 일반적인 인터페이스를 정의합니다. `qiskit.primitives`에 있는 다른 모든 클래스들은 이 base 클래스에서 출발하며, 개발자들은 특정 backend 제공자를 위한 primitive 모델을 개발할 때 이 클래스를 사용해야 합니다. 이 클래스는 현재의 primitive보다 훨씬 복잡하고 특화된 연산을 수행하고 싶은 경우에도 유용할 수 있습니다.

3. `BackendSampler` and `BackendEstimator`

만약 서비스 제공자가 primitive를 직접 제공하지 않는다면, 여러분은 이 클래스를 어떤 backend의 기능을 "묶어서" primitive로 만드는데 사용할 수 있습니다. 유저는 primitive 기반의 인터페이스를 제공하지 않는 제공자를 위해 primitive 스타일의 코드를 작성할 수 있습니다. 이 클래스는 일반적인 Sampler나 Estimator와 똑같이 사용할 수 있지만, 사용할 backend는 직접 지정하여 초기화해야 합니다.

Qiskit Runtime primitives는 클라우드 기반 서비스로서 훨씬 세련된 구현 방법을 제공합니다.

### Get started with primitives

이번에는 primitive를 세팅하는 방법과 primitive에서 이용할 수 있는 옵션, 그것을 프로그램으로 실행하는 방법을 알아봅시다.

#### Get started with Estimator

1. 계정을 설정하기

Qiskit Runtime의 `Estimator`는 실제 운영되는 서비스에 해당하므로, 여러분은 먼저 여러분의 계정을 설정해야 합니다. 그 다음 여러분은 기댓값을 계산하는 데 사용하고 싶은 시뮬레이터나 실제 backend를 선택할 수 있습니다.

만약 아직 계정이 없다면 [Install and set up Qiskit](https://docs.quantum.ibm.com/start/install) 문서를 따라가보세요.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibmq_mumbai")

2. Circuit과 observable을 만들기

앞에서 언급한 바와 같이, `Estimator` primitive에는 circuit과 observable이라는 입력값이 필요합니다.

In [ ]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.quantum_info import SparsePauliOp, random_hermitian

n_qubits = backend.num_qubits

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
observable = SparsePauliOp("Z" * n_qubits)
print(f">>> Observable: {observable.paulis}")

3. Qiskit Runtime Estimator를 설정하기

여기에서는 Qiskit Runtime에서 구현한 `Estimator`를 사용할 것이므로 `qiskit.primitives.Estimator` 대신 `qiskit_ibm_runtime.Estimator`를 설정합니다.

`Estimator`를 설정할 때는 앞에서 선택한 backend를 목표 디바이스(또는 시뮬레이터)로 입력해줘야 합니다. 여러분은 이것을 `session` parameter를 주는 것으로 수행할 수도 있습니다.

In [ ]:
from qiskit_ibm_runtime import Estimator

estimator = Estimator(backend=backend)

4. Estimator를 실행하고 결과값을 얻기

여러분은 `run()` 메소드를 실행해 입력된 circuit과 observable의 기댓값을 계산할 수 있습니다.

In [ ]:
job = estimator.run(circuit, observable)
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

In [ ]:
result = job.result()
print(f">>> {result}")
print(f"  > Expectation value: {result.values[0]}")
print(f"  > Metadata: {result.metadata[0]}")

#### Get started with Sampler

1. 계정을 설정하기

Qiskit Runtime의 `Sampler`는 실제 운영되는 서비스에 해당하므로, 여러분은 먼저 여러분의 계정을 설정해야 합니다. 그 다음 여러분은 준확률분포를 계산하는 데 사용하고 싶은 시뮬레이터나 실제 backend를 선택할 수 있습니다.

만약 아직 계정이 없다면 [Install and set up Qiskit](https://docs.quantum.ibm.com/start/install) 문서를 따라가보세요.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibmq_mumbai")

2. Circuit을 만들기

앞에서 언급한 바와 같이, `Sampler` primitive에는 circuit이라는 입력값이 필요합니다.

In [ ]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.quantum_info import random_hermitian

n_qubits = backend.num_qubits

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
circuit.measure_all()

3. Qiskit Runtime Sampler를 설정하기

여기에서는 Qiskit Runtime에서 구현한 `Sampler`를 사용할 것이므로 `qiskit.primitives.Sampler` 대신 `qiskit_ibm_runtime.Sampler`를 설정합니다.

`Sampler`를 설정할 때는 앞에서 선택한 backend를 목표 디바이스(또는 시뮬레이터)로 입력해줘야 합니다. 여러분은 이것을 `session` parameter를 주는 것으로 수행할 수도 있습니다.

In [ ]:
from qiskit_ibm_runtime import Sampler

sampler = Sampler(backend=backend)

4. Sampler를 실행하고 결과값을 얻기

여러분은 `run()` 메소드를 실행해 입력된 circuit의 준확률분포를 생성할 수 있습니다.

In [ ]:
job = sampler.run(circuit)
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

In [ ]:
result = job.result()
print(f">>> {result}")
print(f"  > Quasi-probability distribution: {result.quasi_dists[0]}")
print(f"  > Metadata: {result.metadata[0]}")